Poniżej znajduje się pełny zestaw materiałów dotyczących `multiprocessing` w Pythonie, obejmujący wstęp teoretyczny, porównania sekwencyjne i równoległe z użyciem przykładów z pomiarem czasu, gotowy do użycia w Jupyter Notebook.

---

### **Materiał o `multiprocessing`**

#### **1. Wprowadzenie**
`multiprocessing` to moduł w Pythonie, który pozwala na tworzenie procesów działających równolegle, wykorzystując wiele rdzeni procesora. Jest używany głównie w aplikacjach CPU-bound, czyli takich, które wymagają intensywnych obliczeń.

---

#### **2. Kluczowe pojęcia**

1. **Proces**:
   - Oddzielna jednostka wykonania, niezależna od innych procesów.
   - Ma własną przestrzeń adresową.

2. **Pool**:
   - Zarządza grupą procesów, umożliwiając efektywne mapowanie funkcji do danych.

3. **Queue**:
   - Kolejka do komunikacji między procesami.

4. **Pipe**:
   - Kanał komunikacyjny między procesami.

5. **Lock**:
   - Narzędzie do synchronizacji procesów.

---

#### **3. Przykłady z kodem**

##### **3.1. Podstawowy przykład: Sekwencyjnie vs równolegle**

###### Sekwencyjne przetwarzanie dużego zakresu danych


In [26]:

%%writefile sequential_primes.py
import time

def is_prime(n):
    """Sprawdza, czy liczba jest pierwsza."""
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

def count_primes_in_range(start, end):
    """Liczy liczby pierwsze w danym zakresie."""
    return sum(1 for i in range(start, end) if is_prime(i))

if __name__ == "__main__":
    start_time = time.time()
    result = count_primes_in_range(2, 10_000_000)  # Zakres do sprawdzenia
    end_time = time.time()
    print(f"Sequential: Found {result} primes in {end_time - start_time:.2f} seconds")



Writing sequential_primes.py



#### Kod równoległy


In [29]:

%%writefile parallel_primes.py
from multiprocessing import Pool
import time

def is_prime(n):
    """Sprawdza, czy liczba jest pierwsza."""
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

def count_primes_in_chunk(chunk):
    """Liczy liczby pierwsze w danym kawałku zakresu."""
    start, end = chunk
    return sum(1 for i in range(start, end) if is_prime(i))

if __name__ == "__main__":
    start_time = time.time()

    # Definiujemy zakres i dzielimy go na kawałki
    RANGE = (2, 10_000_000)
    NUM_PROCESSES = 8
    chunk_size = (RANGE[1] - RANGE[0]) // NUM_PROCESSES
    chunks = [(RANGE[0] + i * chunk_size, RANGE[0] + (i + 1) * chunk_size) for i in range(NUM_PROCESSES)]

    # Tworzymy pulę procesów i przetwarzamy równolegle
    with Pool(NUM_PROCESSES) as pool:
        results = pool.map(count_primes_in_chunk, chunks)

    total_primes = sum(results)  # Sumujemy wyniki z wszystkich procesów
    end_time = time.time()

    print(f"Parallel: Found {total_primes} primes in {end_time - start_time:.2f} seconds")


Overwriting parallel_primes.py


In [28]:
!python sequential_primes.py

Sequential: Found 664579 primes in 157.74 seconds


In [30]:
!python parallel_primes.py

Parallel: Found 664579 primes in 32.52 seconds




---

### **Porównanie wyników**
1. **Sekwencyjny kod**:
   - Przetwarza cały zakres liczb w jednym procesie.
   - Narzut jest minimalny, ale czas wykonania jest długi, ponieważ jedno zadanie wykonuje wszystkie obliczenia.

2. **Równoległy kod**:
   - Dzieli zakres na mniejsze kawałki, które są przetwarzane równolegle przez 4 procesy.
   - Dzięki temu praca jest rozdzielona, co skraca czas wykonania.

---

### **Uruchamianie**
1. Uruchom kod sekwencyjny:
   ```bash
   !python sequential_primes.py
   ```

2. Uruchom kod równoległy:
   ```bash
   !python parallel_primes.py
   ```

---

### **Co pokazuje ten przykład?**
- Równoległość działa najlepiej, gdy:
  - Zadania są intensywne obliczeniowo (np. obliczenia matematyczne).
  - Każde zadanie może być niezależnie przetwarzane (brak współdzielenia danych między procesami).
- W przypadku przetwarzania liczb pierwszych, `multiprocessing` pokazuje znaczące przyspieszenie dzięki równoległemu podziałowi zakresu liczb.

Ten przykład pozwala zobaczyć realne korzyści z równoległego przetwarzania. Przy dużych zakresach różnica czasowa będzie szczególnie widoczna.


---

##### **3.2. Przetwarzanie zadań: Sekwencyjnie vs równolegle**

###### Sekwencyjne generowanie i suma liczb


In [12]:
%%writefile sequential_sum_example.py
import time

def compute_sum(n):
    return sum(range(n))

if __name__ == "__main__":
    start_time = time.time()
    results = [compute_sum(10_000_000) for _ in range(4)]
    print(f"Results: {results}")
    end_time = time.time()
    print(f"Sequential processing time: {end_time - start_time:.2f} seconds")



Writing sequential_sum_example.py


In [13]:
!python sequential_sum_example.py

Results: [49999995000000, 49999995000000, 49999995000000, 49999995000000]
Sequential processing time: 1.33 seconds


###### Równoległe generowanie i suma liczb z użyciem `Process`


In [9]:
%%writefile parallel_sum_example.py
from multiprocessing import Process, Queue  # Importujemy moduł do tworzenia procesów i kolejek do komunikacji między nimi
import time  # Moduł do pomiaru czasu wykonania

def compute_sum(n, queue):
    """
    Funkcja obliczająca sumę liczb od 0 do n-1 i umieszczająca wynik w kolejce.
    
    Args:
        n (int): Liczba, do której obliczana jest suma.
        queue (Queue): Kolejka do komunikacji między procesami.
    """
    queue.put(sum(range(n)))  # Obliczamy sumę i dodajemy ją do kolejki

if __name__ == "__main__":
    start_time = time.time()  # Rozpoczynamy pomiar czasu

    queue = Queue()  # Tworzymy kolejkę do komunikacji między procesami

    # Tworzymy listę procesów, każdy proces wykonuje funkcję compute_sum
    # Funkcji przekazujemy argument n oraz kolejkę, do której zapisuje wynik
    processes = [Process(target=compute_sum, args=(10_000_000, queue)) for _ in range(4)]

    # Uruchamiamy każdy proces
    for p in processes:
        p.start()

    # Czekamy na zakończenie każdego procesu
    for p in processes:
        p.join()

    # Pobieramy wyniki z kolejki
    results = [queue.get() for _ in range(4)]
    print(f"Results: {results}")  # Wyświetlamy wyniki

    end_time = time.time()  # Kończymy pomiar czasu
    print(f"Parallel processing time: {end_time - start_time:.2f} seconds")  # Wyświetlamy czas wykonania


Writing parallel_sum_example.py


In [10]:
!python parallel_sum_example.py

Results: [49999995000000, 49999995000000, 49999995000000, 49999995000000]
Parallel processing time: 0.48 seconds


Kolejka (`Queue`) jest używana tutaj jako mechanizm komunikacji między procesami, ponieważ każdy proces w Pythonie działa w swojej własnej przestrzeni pamięci. Oznacza to, że dane obliczone w jednym procesie nie są automatycznie dostępne dla innych procesów ani dla procesu głównego.

### **Dlaczego potrzebujemy `Queue` w tym przykładzie?**

1. **Oddzielenie przestrzeni pamięci**:
   - Każdy proces w Pythonie tworzony za pomocą `multiprocessing.Process` ma swoją niezależną przestrzeń pamięci.
   - Bez mechanizmu komunikacji, wyniki obliczeń wykonane w procesach potomnych nie mogą być przekazane do procesu głównego.

2. **Przechowywanie wyników**:
   - Kolejka umożliwia procesom potomnym przekazanie swoich wyników do procesu głównego.
   - W tym przypadku każdy proces oblicza sumę liczb od `0` do `n` i zapisuje wynik w kolejce.

3. **Bezpieczeństwo i kolejność**:
   - `Queue` zapewnia bezpieczeństwo wieloprocesowe. Dane umieszczane w kolejce są chronione przed konfliktami między procesami.
   - Dane są pobierane z kolejki w takiej samej kolejności, w jakiej zostały do niej dodane (FIFO – First In, First Out).

---

### **Jak moglibyśmy to zrobić bez `Queue`?**

Alternatywnie, można by wykorzystać inne mechanizmy:
- **`Manager().list`**:
  - Współdzielona lista zarządzana przez `multiprocessing.Manager`.
  - Można by przechowywać wyniki w jednej współdzielonej liście zamiast używać kolejki.
  
- **Zapis wyników do plików**:
  - Procesy mogłyby zapisywać swoje wyniki do plików, które proces główny odczytałby później.
  
- **Wartości zwracane przez `Pool`**:
  - Jeśli używalibyśmy `multiprocessing.Pool`, wyniki obliczeń mogłyby być zwracane bezpośrednio jako lista.

---

Użycie `Queue` w tym przykładzie jest prostym i intuicyjnym sposobem na zbieranie wyników z procesów potomnych w jednym miejscu. Dzięki temu możemy łatwo zrozumieć, jak procesy komunikują się i współpracują ze sobą.


---

#### **4. Alternatywne biblioteki do `multiprocessing`**

1. **`concurrent.futures`**:
   - Moduł wysokopoziomowy, pozwalający na łatwe wykorzystanie wątków i procesów.

2. **`joblib`**:
   - Optymalizowany do przetwarzania dużych danych, szczególnie w analizie danych.

3. **`ray`**:
   - Narzędzie do budowy skalowalnych aplikacji rozproszonych, wspierające obliczenia równoległe.

4. **`dask`**:
   - Framework do przetwarzania równoległego dużych zbiorów danych, doskonały w analizie danych.

5. **`celery`**:
   - Framework do zarządzania zadaniami asynchronicznymi, używany w aplikacjach webowych i przetwarzaniu w tle.

